In [40]:
import pandas as pd
import pickle
import datetime

In [108]:
with open('mta_extract.pickle','rb') as read_file:
    mta_data = pickle.load(read_file)

In [109]:
#Convert DATE to date format
mta_data['DAY'] = pd.to_datetime(mta_data['DATE'])
mta_data.columns = [column.strip() for column in mta_data.columns]

In [110]:
#Get rid of duplicates and Recover Audit entries
by_day = mta_data[(mta_data['DESC'] == 'REGULAR')]

In [111]:
#Create a data frame that looks at each turnstile and identifies the lowest recorded entries for the day. 
#We assume that the first count of the day is the beginning of the day/end of previous day
by_day = by_day.sort_values(['C/A','UNIT','SCP','DAY','ENTRIES'], ascending = True).groupby(['C/A','UNIT','SCP','DAY']).head(1)

In [112]:
by_day.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/09/2018,00:00:00,REGULAR,6649975,2254182,2018-06-09
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2018,00:00:00,REGULAR,6650723,2254407,2018-06-10
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/11/2018,00:00:00,REGULAR,6651277,2254564,2018-06-11
18,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2018,00:00:00,REGULAR,6652785,2255022,2018-06-12
24,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/13/2018,00:00:00,REGULAR,6654264,2255553,2018-06-13


In [113]:
#Set DAY to previous day. This allows us to assign today's lowest count - yesterday's lowest count = yesterday's total entries.
by_day['DAY'] = by_day['DAY'] - pd.Timedelta(1,'D')
by_day['COUNT'] = by_day.sort_values(['C/A','UNIT','SCP','DAY']).groupby(['C/A','UNIT','SCP'])['ENTRIES'].diff()
by_day['COUNT OF EXITS'] = by_day.sort_values(['C/A','UNIT','SCP','DAY']).groupby(['C/A','UNIT','SCP'])['EXITS'].diff()

In [114]:
#get rid of NaN and correct negative counters

by_day.dropna(axis = 0,inplace = True)
by_day = by_day.sort_values(['C/A','UNIT','SCP','LINENAME','DIVISION','STATION','DAY'])

In [115]:
# Create function to correct negative counters and counters that defy logic. i.e. more than 90000 people through the turnstile
def data_cleanse(row, column):
    counter = row[column]
    if counter < 0:
        counter = -counter
    if counter > 90000:
        counter = 0
    return counter

In [116]:
by_day['DAILY COUNTS'] = by_day.apply(data_cleanse, axis = 1, column = 'COUNT')
by_day['DAILY EXITS'] = by_day.apply(data_cleanse, axis = 1, column = 'COUNT OF EXITS')


In [117]:
by_day['NAME OF DAY'] = by_day['DAY'].dt.weekday_name
by_day['WEEK NUMBER'] = by_day['DAY'].dt.week

In [118]:
def fixed_week(row):
    week = row['WEEK NUMBER']
    if row['NAME OF DAY'] == 'Saturday':
        week += 1
    elif row['NAME OF DAY'] == 'Sunday':
        week += 1
    return week

In [120]:
by_day['WEEK NUMBER'] = by_day.apply(fixed_week, axis=1)

In [129]:
by_day = by_day.drop(['TIME','DESC','ENTRIES','EXITS','COUNT','COUNT OF EXITS'], axis = 1, errors = 'ignore')

In [122]:
with open('mta_clean_by_day.pickle', 'wb') as to_write:
    pickle.dump(by_day, to_write)